<a href="https://colab.research.google.com/github/marcehluna/DIIA---Clase-5/blob/main/GCP_RAG_Custom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Custom RAG con GCP

## Introducción y contexto
A lo largo de las sesiones anteriores, hemos explorado en profundidad los sistemas de Generación Aumentada por Recuperación (RAG), una arquitectura fundamental en el campo del Procesamiento del Lenguaje Natural (PLN) que permite a los modelos de lenguaje de gran tamaño (LLMs) generar respuestas más precisas y contextualizadas. Hemos aprendido a implementar y a interactuar con sistemas RAG utilizando servicios mayoritariamente autogestionados dentro del ecosistema de Google Cloud.

Hasta ahora, nuestro enfoque se ha centrado en utilizar componentes pre-construidos donde la fase de indexación —el proceso de ingestar y preparar los datos para que el sistema de recuperación pueda actuar sobre ellos— se realizaba de forma casi automática. Hemos tratado con servicios totalmente gestionados y administrados, lo que nos ha permitido concentrarnos en la interacción entre el recuperador y el generador.

Sin embargo, el mundo real necesita de soluciones más detalladas y precisas. La vasta mayoría de la información valiosa para las empresas y organizaciones se encuentra "atrapada" en formatos complejos como los documentos PDF. Estos archivos no son solo contenedores de texto; son un lienzo que combina texto, imágenes, tablas, columnas y una estructura visual que es trivial para un humano, pero inmensamente compleja para una máquina.
Este cuaderno marca el siguiente paso en nuestra formación como especialistas en Deep Learning: nos adentraremos en el desafío de construir nosotros mismos el pipeline de indexación.

## Descripción del problema

Nuestro objetivo es procesar un documento PDF desde cero, extraer su contenido de manera inteligente y prepararlo para ser vectorizado e insertado en una base de datos vectorial. Este proceso es la piedra angular de cualquier sistema RAG robusto y escalable.

Para lograrlo, no nos basta con una simple extracción de texto. Necesitamos entender la estructura del documento. Aquí es donde entran en juego dos tecnologías críticas:

* Análisis de Disposición de Página (Layout Analysis): Esta parte del procesado nos especifica dónde y de qué tipo es el contenido de la página. Identifica párrafos, títulos, encabezados, pies de página, tablas y columnas. Entender el layout es crucial porque el orden y la agrupación del texto definen su contexto semántico. Por ejemplo, un texto en una columna debe leerse de arriba abajo antes de pasar a la siguiente columna, y los datos dentro de una tabla tienen una relación entre sí que se perdería con una lectura lineal.

* Reconocimiento Óptico de Caracteres (OCR): Es el proceso de convertir imágenes de texto (como las que se encuentran en un PDF escaneado) en texto real que una máquina puede leer y procesar. Es la tarea siguiente a realizar una vez el _layout analysis_ ha hecho su parte.

  ![](https://miro.medium.com/v2/resize:fit:1400/0*9WSDnnY0aI05R_cq.png)

Para esta tarea, utilizaremos el servicio `PPStructureV3` de `PaddlePaddle`. Este no es un servicio de OCR convencional; está diseñado específicamente para el análisis inteligente de documentos y ofrece capacidades de vanguardia:

1. Modelos Pre-entrenados: Utiliza modelos de Deep Learning entrenados en miles de millones de documentos, capaces de entender una variedad de layouts complejos sin necesidad de entrenamiento adicional.

2. Identificación de Bloques: El servicio no devuelve una masa de texto desordenada. En su lugar, segmenta el documento en una estructura jerárquica: Páginas -> Bloques -> Párrafos -> Palabras -> Símbolos. Cada uno de estos elementos viene con su contenido de texto y sus coordenadas (bounding box).

3. Manejo de Entidades: Es capaz de identificar entidades específicas como tablas, listas y otros elementos estructurales, permitiendo una extracción de datos mucho más granular y precisa.

Al utilizar `PaddlePaddle`, podremos descomponer nuestro PDF en fragmentos de texto lógicos y coherentes. Estos fragmentos (o chunks) serán las unidades que posteriormente convertiremos en vectores para alimentar nuestra base de datos vectorial, asegurando que cada vector represente una idea o un concepto semánticamente cohesivo.

A continuación, procederemos con la implementación práctica, pero previo a ellos es importante asegurarse de que tenemos habilitada la GPU en Colab.





## Paso 0: Instalación de librerías

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%%script bash
# Install parallel
sudo apt update && sudo apt install ghostscript && sudo apt install parallel

# Install uv
pip install uv

# Install PaddlePaddle GPU 3.0.0 (for CUDA 12.6)
uv pip install --pre paddlepaddle-gpu -i https://www.paddlepaddle.org.cn/packages/nightly/cu126/

# Install PaddleOCR version that includes PPStructureV3
uv pip install paddleocr==3.1.0 sentence-transformers einops timm pillow qdrant-client

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists...
Building dependency tree...
Reading state information...
36 packages can be upgraded. Run 'apt list --upgradable' to see them.
Reading package lists...
Building dependency tree...
Reading state information...
ghostscript is already the newest version (9.55.0~dfsg1



W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)




Using Python 3.11.13 environment at: /usr
Resolved 29 packages in 5.15s
Uninstalled 13 packages in 53ms
Installed 13 packages in 74ms
 - nvidia-cublas-cu12==12.4.5.8
 + nvidia-cublas-cu12==12.6.4.1
 - nvidia-cuda-cupti-cu12==12.4.127
 + nvidia-cuda-cupti-cu12==12.6.80
 - nvidia-cuda-nvrtc-cu12==12.4.127
 + nvidia-cuda-nvrtc-cu12==12.6.77
 - nvidia-cuda-runtime-cu12==12.4.127
 + nvidia-cuda-runtime-cu12==12.6.77
 - nvidia-cudnn-cu12==9.1.0.70
 + nvidia-cudnn-cu12==9.5.1.17
 - nvidia-cufft-cu12==11.2.1.3
 + nvidia-cufft-cu12==11.3.0.4
 - nvidia-curand-cu12==10.3.5.147
 + nvidia-curand-cu12==10.3.7.77
 - nvidia-cusolver-cu12==11.6.1.9
 + nvidia-cusolver-cu12==11.7.1.2
 - nvidia-cusparse-cu12==12.3.1.170
 + nvidia-cusparse-cu12==12.5.4.2
 - nvidia-cusparselt-cu12==0.6.2
 + nvidia-cusparselt-cu12==0

## Paso 1: Lectura del PDF

En primer lugar, vamos a definir un método que nos permita, a partir de una determinada ruta en la que especifiquemos un archivo `.pdf`, dividirlo en páginas independientes, y convertir cada una de ellas a `.jpg`.

In [4]:
import os
import subprocess
import logging as log
from typing import List, Dict

log.basicConfig(level=log.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def convert_pdf_to_images(pdf_path: str, output_dir: str = "images", dpi: int = 300) -> Dict[str, List[str]]:

    filepaths = []
    if os.path.isfile(pdf_path) and pdf_path.lower().endswith(".pdf"):
        filepaths.append(pdf_path)
    elif os.path.isdir(pdf_path):
        for root, _, filenames in os.walk(pdf_path):
            for filename in filenames:
                if filename.lower().endswith(".pdf"):
                    filepaths.append(os.path.join(root, filename))
    else:
        raise FileNotFoundError(f"No valid PDF files found at the specified path: {pdf_path}")

    if not filepaths:
        raise FileNotFoundError(f"No PDF files were found to process in {pdf_path}.")

    file2image = {}
    for filepath in filepaths:
        name = os.path.splitext(os.path.basename(filepath))[0]
        output_path = os.path.join(output_dir, name)

        if os.path.isdir(output_path) and os.listdir(output_path):
            log.info(f"Images for '{name}' already exist. Skipping conversion.")
            file2image[name] = sorted([os.path.join(output_path, img) for img in os.listdir(output_path) if img.endswith(".jpg")])
            continue

        os.makedirs(output_path, exist_ok=True)
        log.info(f"Converting '{name}' to images at {dpi} DPI...")
        cmd = [
            "gs", "-dNOPAUSE", "-dBATCH", "-sDEVICE=jpeg",
            f"-r{dpi}", f"-sOutputFile='{os.path.join(output_path, 'page_%03d.jpg')}'",
            f"'{filepath}'",
        ]
        try:
            subprocess.run(" ".join(cmd), check=True, capture_output=True, text=True, shell=True)
        except subprocess.CalledProcessError as e:
            log.error(f"Ghostscript failed for {filepath}: {e.stderr}")
            raise RuntimeError(f"PDF to image conversion failed for {filepath}: {e}")

        file2image[name] = sorted([os.path.join(output_path, img) for img in os.listdir(output_path) if img.endswith(".jpg")])
    return file2image

Descargamos un `.pdf` como referencia:

In [5]:
import requests
import urllib.parse

#url = 'https://arxiv.org/pdf/2505.09388'
url = 'https://bibliotecanautica.appweb.ar/_material//00000200_Manuales%2C%20Construcci%C3%B3n%2C%20Navegaci%C3%B3n/Manuales%20y%20Cursos/Timonel%20Motor%20F%C3%A1cil%20%28Hern%C3%A1n%20Biasotti%29.pdf'
parsed_url = urllib.parse.unquote(os.path.basename(url))
fn = os.path.splitext(parsed_url)[0]


# Use stream=True to not load the whole content into memory
with requests.get(url, stream=True, timeout=10) as r:
    r.raise_for_status()

    # Open the file in binary write mode
    with open(f"{fn}.pdf", 'wb') as f:
        # Download the file in chunks
        for chunk in r.iter_content(chunk_size=8192): # 8KB chunks
            f.write(chunk)

print(f"Successfully downloaded and saved as '{fn}.pdf'")

Successfully downloaded and saved as 'Timonel Motor Fácil (Hernán Biasotti).pdf'


Una vez definido el método, nos aseguramos de que generamos un directorio donde las imágenes puedan ser almacenadas, y lo empleamos:

In [ ]:
# Especificamos la ruta donde se aloja el PDF
pdf_path = f"{fn}.pdf"
# Nos aseguramos de que generamos un directorio para las imágenes
output_dir = 'images'
os.makedirs(output_dir, exist_ok=True)
# Ejecutamos el método construído
doc2images = convert_pdf_to_images(pdf_path, output_dir)

## Paso 2: Procesamiento `OCR`

Como ya indicamos, `PaddlePaddle` dispone de sistemas de alto rendimiento para nuestro propósito, incluyendo complejos _pipelines_ que distinguen entre tipos de objetos, posicionamiento, idiomas, entre otros:

![](https://raw.githubusercontent.com/cuicheng01/PaddleX_doc_images/refs/heads/main/images/paddleocr/PP-StructureV3/algorithm_ppstructurev3.png)

Dado que, además, es un servicio de código abierto, podemos emplearlo cómodamente en nuestras soluciones. A continuación, mostramos cómo hacerlo:

In [ ]:
from paddleocr import PPStructureV3

pipeline = PPStructureV3(
    layout_detection_model_name="PP-DocLayout_plus-L",
    text_recognition_model_name="PP-OCRv5_server_rec",
    use_doc_orientation_classify=True,
    use_doc_unwarping=True,
    use_textline_orientation=True,
    device="gpu",
)

output = pipeline.predict(doc2images[fn])

/usr/local/lib/python3.11/dist-packages/paddle/utils/cpp_extension/extension_utils.py:717: UserWarning: No ccache found. Please be aware that recompiling all source files may be required. You can download and install ccache from: https://github.com/ccache/ccache/blob/master/doc/INSTALL.md
  warnings.warn(warning_message)
Creating model: ('PP-LCNet_x1_0_doc_ori', None)
Using official model (PP-LCNet_x1_0_doc_ori), the model files will be automatically downloaded and saved in /root/.paddlex/official_models.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models o

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

Creating model: ('UVDoc', None)
Using official model (UVDoc), the model files will be automatically downloaded and saved in /root/.paddlex/official_models.


Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

Creating model: ('PP-DocBlockLayout', None)
Using official model (PP-DocBlockLayout), the model files will be automatically downloaded and saved in /root/.paddlex/official_models.


...

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

Creating model: ('PP-OCRv5_server_det', None)
Using official model (PP-OCRv5_server_det), the model files will be automatically downloaded and saved in /root/.paddlex/official_models.


Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

Creating model: ('PP-OCRv5_server_rec', None)
Using official model (PP-OCRv5_server_rec), the model files will be automatically downloaded and saved in /root/.paddlex/official_models.


Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

## Paso 3: Análisis del resultado e indexación

El resultado de este análisis es tan detallado como el proceso en sí. En concreto, la salida del `pipeline` será una lista de diccionarios, en la que nos interesa particularmente el elemento `parsing_res_list`. En él, tendremos toda la información disponible; en particular, dicha información estará tipificada en los siguientes conjuntos:

* header
* doc_title
* text
* paragraph_title
* figure_title
* image
* footer
* content
* number
* table
* footnote
* chart



A fin de encontrar un buen equilibrio entre eficacia y eficiencia, analizaremos los contenidos más esenciales, que son `text`, `content`, `paragraph_title`, `image`, `table` y `chart`. Teniendo en consideración que nuestro objetivo final es vectorizar nuestro documento (previo paso de indexación o _chunking_), la estrategia que tomamos es:

1. Para los elementos de texto (que incluye a `text`, `content` y `paragraph_title`), simplemente tomamos el contenido y lo vectorizamos. Un ejemplo de lo que el proceso `OCR` nos devuelve sobre estos elementos es:

  ```json
  {
    'label': 'text',
    'order_label': 'normal_text',
    'bbox': [166, 952, 1633, 1192],
    'content': 'An Architecture for Building Agentic Applications in the Enterprise ',
    'width': np.float32(1466.4224),
    'height': np.float32(239.9646),
    'area': 351889.4552630186,
    'num_of_lines': 2,
    'image': None,
    'index': 0,
    'order_index': 2,
    'text_line_width': np.float64(1328.0),
    'text_line_height': np.float64(121.5),
    'child_blocks': [],
    'direction': 'horizontal',
    'secondary_direction': 'vertical',
    #...
  }
  ```
2. Para elementos de tipo tabla, tendríamos la opción de vectorizar el texto resultado de nuestro sistema OCR, o bien vectorizar directamente la tabla como una imagen. En este caso, tomaremos la priemra aproximación al problema por cuestiones de eficiencia, aunque para tablas muy complejas podremos siempre optar por la segunda. Nuestro proceso nos retorna de estos objetos lo siguiente:

  ```json
    {
      'label': 'table',
      'order_label': 'vision',
      'bbox': [297, 497, 1498, 1953],
      'content': '<html><body><table><tbody><tr><td>Developer or Provider</     td><td>Model or Product</td><td>Release Date</td><td>Description</td></     tr><tr><td>OpenAl</td><td>GPT-3</td><td>May 2020</  td><td>175billionparameter   LLMwith2048token contextwindow</td></  tr><tr><td>OpenAl</td><td>ChatGPT</   td><td>November 2022</td><td>Consumer   chatbotapplication,poweredby GPT-3.   5Turbo</td></tr><tr><td>Microsoft   Azure</td><td>OpenAl Service</ td><td>January  2023</td><td>Managed   serviceofferingLLMs fromOpenAI</td></ tr><tr><td>Amazon Web  Services</ td><td>Bedrock</td><td>September 2023</ td><td>Managed serviceoffering   LLMs from various developers</td></  tr><tr><td>Dataiku</td><td>LLMMesh</    td><td>September 2023</ td><td>CommercialLLMMeshoffering forconnecting to   LLMs   and buildinq agentic  applications in the enterprise</td></    tr><tr><td>Databricks</td><td>DBRX</  td><td>March 2024</ td><td>Open-weiqhts  mixture ofexperts model with 132B   total parameters  and 32k-token input context   window,licensed forcommercial  use</td></  tr><tr><td>Meta</td><td>LLaMA3 (8B,70B) </td><td>April2024</  td><td>UpdatedLLMwith4096-tokeninputcontext window,  withupdated     licenseallowingcertain commercial uses</td></tr><tr><td>Mistral</     td><td>Mixtral 8x22B</td><td>April 2024</td><td>Open-weights mixture    ofexperts   model with up to141Bparameters and64k-inputcontext window,  licensed  forcommercial use</td></tr><tr><td>OpenAl</td><td>GPT-40</   td><td>May 2024</  td><td>Multimodal LLM supporting voice-to-voice  generation  and128k-token input  context window</td></tr><tr><td>OpenAl</  td><td>01</ td><td>September 2024</ td><td>Reasoningmodel withbuilt-in  chain-of-thought  for complex scientificand  mathematical problems</td></  tr><tr><td>DeepSeek</ td><td>R1</td><td>January   2025</td><td>Open-source  reasoning model (MIT  license) optimized for math,  coding, and logic</td></ tr></tbody></table></  body></html>',
      'width': np.float32(1200.5344),
      'height': np.float32(1456.0574),
      'area': 1748046.9994115233,
      'num_of_lines': 1,
      'image': {'path': 'imgs/img_in_table_box_297_497_1498_1953.jpg',
       'img': <PIL.Image.Image image mode=RGB size=1201x1456>},
      'index': 0,
      'child_blocks': [],
      'direction': 'horizontal',
      'secondary_direction': 'vertical',
      #...
    }
  ```

3. Para elementos tipo imagen (entre los que están `image` y `chart`), directamente accederemos al atributo `image` para tomar la imagen como _array_ numérico:

  ```json
  {
  'label': 'image',
   'order_label': 'vision',
   'bbox': [121, 1709, 1676, 2439],
   'content': 'The Universal Al Platform T \nControl Agents Enterprise  Orchestration Continuous Optimization Central Govermance Multi-Agents \nAgent  Observabillity Strategic Oversight Multi-Models \nAnalytics Models   \nSecurity & Guardrails Agent Performance Risk Monitoring \nCreate Agents   Knowledge Worker Developer Data & Al Infrastructure Third-Party Agents & Tools  A aws W ',
   'seg_start_coordinate': np.float64(694.0),
   'seg_end_coordinate': np.float64(1548.0),
   'width': np.float32(1554.9723),
   'height': np.float32(729.7499),
   'area': 1134740.8388400525,
   'num_of_lines': 1,
   'image': {'path': 'imgs/img_in_image_box_121_1709_1676_2439.jpg',
    'img': <PIL.Image.Image image mode=RGB size=1555x730>},
   'index': 3,
   'order_index': None,
   'text_line_width': np.float64(614.4166666666666),
   'text_line_height': np.float64(34.25),
   'child_blocks': [],
   'direction': 'horizontal',
   'secondary_direction': 'vertical',
   #...
   }
  ```

  Observamos en particular que el proceso `OCR` hace un esfuerzo por proporcionar información textual sobre imágenes, gráficas y tablas.

Vamos a proceder ahora a almacenar la información que estrictamente necesitamos del proceso `OCR`. Concretamente, para hacer una indexación completa, necesitaremos la información que mostramos en la siguiente estructura de datos:

In [ ]:
# Dependencias
from pydantic import BaseModel, Field, field_validator, ConfigDict
from PIL import Image
from typing import List, Union, Dict, Any, Optional
import numpy as np

# Definimos una estructura que contenga nuestros elementos atómicos
class DocumentElement(BaseModel):
    """
    Representa un único elemento atómico detectado en un documento, como un
    bloque de texto, un título, una tabla o una imagen.

    Esta clase utiliza Pydantic para validar los datos y asegurar que cada
    elemento tenga la estructura correcta.
    """

    document_id: str          # Identificador del documento al que pertenece.
    page_number: int          # Número de la página donde se encuentra el elemento.
    layout_index: int         # Índice del elemento dentro del análisis del layout.
    label: str                # Etiqueta específica del modelo (ej: 'text', 'title', 'table').
    region_label: str         # Etiqueta de la región a la que pertenece (ej: 'header', 'body').

    # El cuadro delimitador (bounding box) [x1, y1, x2, y2].
    # Field(...) indica que es obligatorio. min_items y max_items aseguran que siempre tenga 4 coordenadas.
    bbox: List[int] = Field(..., min_items=4, max_items=4)

    # El contenido del elemento. Puede ser texto (str) o una imagen (np.array).
    content: Union[str, List[float], List[List[float]], List[List[List[float]]]]

Almacenamos ahora en una lista los resultados:

In [ ]:
# Creamos una lista vacía donde iremos acumulando los `chunks`
l = []

# Iteramos sobre cada página
for page_number, item in enumerate(output):
    # Iteramos sobre cada elemento de salida
    for elem in item['parsing_res_list']:
        # Si la entidad detectada es una imagen, la almacenamos como array de números
        if elem.label in ['image', 'chart']:
            l.append(
                DocumentElement(
                    document_id=fn,
                    page_number=page_number,
                    layout_index=elem.index,
                    label=elem.label,
                    region_label=elem.order_label,
                    bbox=elem.bbox,
                    content=np.array(elem.image['img']).tolist()
                )
            )
        elif elem.label in ['text', 'content', 'paragraph_title', 'table']:
            l.append(
                DocumentElement(
                    document_id=fn,
                    page_number=page_number,
                    layout_index=elem.index,
                    label=elem.label,
                    region_label=elem.order_label,
                    bbox=elem.bbox,
                    content=elem.content
                )
            )
        else:
            continue

# Guardamos el objeto resultante
with open(f'{fn}_ocr_result.jsonl', 'w', encoding='utf-8') as f:
    for line in l:
        json_line = line.model_dump_json()
        f.write(json_line + '\n')

## Paso 4: Vectorización

El siguiente paso es generar una estrategia para _embeber_ nuestra información (bien sea de tipo textual o imágenes) en una serie de números, con la finalidad de poder establecer criterios de comparación mediante ciertas métricas, como la distancia coseno.

![](https://www.mlflow.org/docs/3.0.1/assets/images/sentence-transformers-architecture-b83485a83e698e3e1576f44024570e81.png)


### Contexto: Sentence Transformers


Una de las familias de modelos más populares para dicha tarea se conocen como **sentence-transformers**, y será [una variante de ellos](https://huggingface.co/jinaai/jina-clip-v2) los que empleemos en nuestro propósito.

A fin de tener una mínima comprensión del funcionamiento de las variantes más clásicas de estos modelos, y sin intención de entrar en excesivos detalles técnicos, procedemos a ilustrar algunos detalles de su funcionamiento. Tradicionalmente, los primeros modelos de procesamiento de lenguaje natural que representaban lenguaje, estaban basados en frecuencias de determinadas palabras de un _vocabulario_ específico, basado en un _corpus_ de entrenamiento que se analiza con criterios estadísticos.

![](https://www.pinecone.io/_next/image/?url=https%3A%2F%2Fcdn.sanity.io%2Fimages%2Fvr8gru94%2Fproduction%2F96a71c0c08ba669c5a5a3af564cbffee81af9c6d-1920x1080.png&w=1920&q=75)

Si bien estos modelos son sencillos de entrenar y poseen facilidad de convergencia _local_, somos nosotros los que estamos imponiendo un esquema de vectorización fijo (dando lugar a los modelos conocidos como _sparse_), sin dar margen de optimización al modelo a que **elija de forma abstracta qué variables emplear**. Estos últimos modelos se conocen como _densos_. En ellos, la vectorización no consiste directamente en criterios palpables, si no que en su lugar el modelo decide qué criterios (como caja negra) emplear para asignar esos números. Este entrenamiento es mucho más complejo y requiere de muchísimos más datos, pero da lugar a modelos más poderosos.

En cierto sentido, podemos considerar entonces que disponemos de una _caja negra_ que _transforma_ nuestros textos en vectores, que son comparables mediante criterios como la similitud del coseno. Este principio de comparación se puede extender a otras modalidades de datos como imágenes o audios, dando lugar a modelos **multimodales**.

![](https://www.pinecone.io/_next/image/?url=https%3A%2F%2Fcdn.sanity.io%2Fimages%2Fvr8gru94%2Fproduction%2Fd0e73377d123ccf0e910f4b971f6cb06bb87f200-1920x1080.png&w=1920&q=75)

Esto nos va a permitir, en última instancia, obtener la similitud entre la pregunta de un usuario en un chat con cualquier tipo de información que hemos obtenido de nuestros documentos, como tablas o imágenes, no sólo texto.

Finalmente, existe un último punto, que consiste precisamente en la elección de la longitud de esos vectores. Si bien damos _libertad_ al modelo durante el entrenamiento para decidir cómo configurar las variables numéricas, le imponemos que sea una cantidad fija. Este es un limitante bastante grande de cara a, por ejemplo, optimizar el equilibrio entre rendimiento y velocidad de las soluciones, para lo que surgió una estrategia adaptativa llamada **Matryoshka Representation Learning**, que permitía mantener una gran cantidad de precisión en ciertas tareas, acortando la longitud de los vectores, pudiendo así priorizar según nuestro caso qué configuración elegir.

![](https://miro.medium.com/v2/resize:fit:782/1*MAwYcyyo2mFC02bGA5-Xzw.png)

Dado que la comparación de vectores más cortos es más rápida, para soluciones que necesiten una velocidad extrema, priorizaremos acortar la longitud de los mismos, y cuando la precisión sea vital, haremos lo contrario. Todos estos avances los vamos a poner en funcionamiento en nuestra implementación.


### Estrategia de _chunking_

Veamos ahora cómo segmentar nuestra información basada en el planteamiento inicial. PAra ello, trataremos de forma independiente las imágenes/gráficos/tablas, y por otro lado el puro "texto". Con respecto a los primeros, los indexaremos de forma independiente y almacenaremos como un `string` tipo `base64`, que es como los modelos de lenguaje típicamente consumen estos elementos, y para el texto haremos una estrategia de _semantic chunking_, que ya describimos en otra sesión.

Inicialmente, cargamos las dependencias, definimos las variables (que podéis sentiros libres de modificar para ver el comportamiento del sistema), cargamos el resultado del OCR, y finalmente inicializamos el modelo que vectoriza nuestra data:

In [ ]:
# Dependencias
import json
import io
import base64
from pydantic import BaseModel, Field, field_validator, ConfigDict
from PIL import Image
from typing import List, Union, Dict, Any, Optional
import torch
from sentence_transformers import SentenceTransformer

# Definimos las variables
truncate_dim = 256
min_chunk_size = 128
max_chunk_size = 512
similarity_threshold = 0.3
format = 'PNG'

# Leemos el resultado del OCR
l = []
with open(f"{fn}_ocr_result.jsonl", 'r', encoding='utf-8') as f:
  for line in f:
      json_line = json.loads(line)
      l.append(DocumentElement(**json_line))

# Definimos una estructura que contenga nuestros elementos atómicos
class Chunk(BaseModel):
    document_id: str          # Identificador del documento al que pertenece.
    label: str                # Etiqueta específica del modelo (ej: 'text', 'title', 'table').
    region_label: str         # Etiqueta de la región a la que pertenece (ej: 'header', 'body').
    embedding: List[float]    # Para construir el modelo, usamos texto o imagen según convenga
    content: str              # Usamos base64 en caso sea imagen

# Cargamos el modelo
model = SentenceTransformer(
    'jinaai/jina-clip-v2', trust_remote_code=True, truncate_dim=truncate_dim
)

Es ahora cuando definimos la estrategia para crear nuestros `chunks` de datos:

In [ ]:
chunks = []
text = ''
for elem in l:
    if elem.label in ['image', 'chart']:
        # Codificamos la imagen
        img = Image.fromarray(np.array(elem.content, dtype=np.uint8))
        emb = model.encode(img, normalize_embeddings=True)
        # Convertimos la imagen a base64 (la forma en la que los LLMs consumen imágenes)
        buffered = io.BytesIO()
        img.save(buffered, format=format)
        img_str = base64.b64encode(buffered.getvalue()).decode("utf-8")
        mime_type = f"image/{format.lower()}"
        base64_image = f"data:{mime_type};base64,{img_str}"
        # Guardamos un chunk independiente al flujo de texto
        chunks.append(
            Chunk(
                document_id=elem.document_id,
                label=elem.label,
                region_label=elem.region_label,
                embedding=emb.tolist(),
                content=base64_image,
            )
        )
    elif elem.label in ['text', 'paragraph_title', 'content']:
        tokenized_text_len = len(model.tokenizer.encode(text))
        # Si el texto tiene una longitud menor a la que contemplamos como posible
        if tokenized_text_len<min_chunk_size:
            text += ' ' + elem.content
        # Si el texto se encuentra en una longitud candidata para ser un chunk
        elif ((tokenized_text_len>=min_chunk_size) & (tokenized_text_len<=max_chunk_size)):
            embs = model.encode([text, elem.content], normalize_embeddings=True)
            # Si la similitud es demasiado cercana
            if float(np.dot(embs[0], embs[1]))>=similarity_threshold:
                text += ' ' + elem.content
            # Si los textos no se parecen lo suficiente
            else:
                # Guardamos el texto del chunk hasta la fecha
                chunks.append(
                    Chunk(
                        document_id=elem.document_id,
                        label='text',
                        region_label='text',
                        embedding=embs[0].tolist(),
                        content=text,
                    )
                )
                # Empezamos nuevo chunk
                text = elem.content
        # Si el texto ya excede la longitud establecida
        else:
            # Guardamos el texto del chunk hasta la fecha
            emb = model.encode(text, normalize_embeddings=True)
            chunks.append(
                Chunk(
                    document_id=elem.document_id,
                    label='text',
                    region_label='text',
                    embedding=emb.tolist(),
                    content=text,
                )
            )
            # Empezamos nuevo chunk
            text = elem.content

## Paso 5: Indexación en base de datos

Finalmente, emplearemos un tipo especial de base de datos, que se ha popularizado mucho con los recientea avances de IA generativa, que son las **bases de datos de vectores**. Éstas implementan algoritmos específicos para búsqueda eficiente de información que viene almacenada en el formato que hemos preparado.

In [ ]:
# Dependencias
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams

# Definimos un cliente local en memoria
client = QdrantClient(":memory:")

# Creamos una nueva colección para albergar nuestros documentos
client.create_collection(
    collection_name=f"{fn}_collection",
    vectors_config=VectorParams(size=truncate_dim, distance=Distance.COSINE),
)